In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy import fftpack
from scipy import signal
from numericalunits import µ0, NA, kB, hbar, mm, cm, m, s, ms, us, Hz, kHz, MHz
from numericalunits import T, K, J, g, mol, A, ohm, W, N, kg, mV, V, eV, uV, mT, uT

from FID_simulation import StorageRingMagnet, FixedProbe, Noise

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm as mcolors
import matplotlib
from matplotlib.gridspec import GridSpec

In [7]:
def run_sim(multipole, strength):
    external_field = StorageRingMagnet( )
    external_field.set_strength_at_1cm(multipole, strength)
    
    nmr_probe = FixedProbe(B_field=external_field, N_cells=10000)

    B0 = 1.45*T
    cmap = "seismic"

    title = external_field.strength_to_str(multipole, strength)
    
    ppm = (nmr_probe.cells_B0-B0)/B0*1e6
    dB = np.max(np.abs(ppm))

    fig = plt.figure()
    gs = GridSpec(5, 6)
    ax1 = plt.subplot(gs[0:2, :])
    ax2 = plt.subplot(gs[2:, :3])
    ax3 = plt.subplot(gs[2:, 3:], projection='3d')

    fig.suptitle(title)

    ax1.scatter(nmr_probe.cells_z/mm, nmr_probe.cells_y/mm, c=ppm, cmap=cmap, vmin=-dB, vmax=dB, s=3)
    ax1.set_aspect("equal")
    ax1.set_xlabel("z/mm")
    ax1.set_ylabel("y/mm")
    ax1.set_xlim(-15, 15)

    sc = ax2.scatter(nmr_probe.cells_x/mm, nmr_probe.cells_y/mm, c=ppm, cmap=cmap, vmin=-dB, vmax=dB, s=3)
    ax2.set_aspect("equal")
    ax2.set_xlabel("x/mm")
    ax2.set_ylabel("y/mm")
    plt.colorbar(sc, ax=ax2).set_label(r"$\Delta B\, /\, \mathrm{ppm}$")

    sc = ax3.scatter(nmr_probe.cells_x/mm, nmr_probe.cells_z/mm, nmr_probe.cells_y/mm, c=ppm, cmap=cmap, vmin=-dB, vmax=dB, s=3)
    ax3.set_xlim(-5,5)
    ax3.set_ylim(-5,5)
    ax3.set_zlim(-5,5)
    ax3.set_xlabel("x/mm")
    ax3.set_ylabel("z/mm")
    ax3.set_zlabel("y/mm")

    plt.tight_layout()
    fig.savefig("./plots/multipole_plot_%d.png"%int(multipole), dpi=200)
    fig.savefig("./plots/multipole_plot_%d.pdf"%int(multipole), bbox_inches="tight")
    plt.close()

In [9]:
for multi in range(1,24):
    run_sim(multi, 5e-6)